In [25]:
from tools import *

encoder = read_object('encoder_calendar.pkl', Path('inference') / 'firepoint' / '2x2' / 'train' / 'Encoder')

In [3]:
import numpy as np
from scipy import stats

class ModelAggregator:
    def __init__(self, task_type, weights_for_model):
        """
        Initialize the ModelAggregator.

        Parameters:
        - task_type: 'classification' or 'regression'.
        - weights_for_model: List of weights corresponding to each model.
        """
        self.task_type = task_type
        self.weights_for_model = np.array(weights_for_model)

    def aggregate_predictions(self, predictions_list):
        """
        Aggregate predictions from multiple models with weights.

        Parameters:
        - predictions_list: List of predictions from each model.

        Returns:
        - Aggregated predictions.
        """
        predictions_array = np.array(predictions_list)
        
        if self.task_type == 'classification':
            # Weighted vote for classification
            unique_classes = np.unique(predictions_array)
            weighted_votes = np.zeros((len(unique_classes), predictions_array.shape[1]))

            for i, cls in enumerate(unique_classes):
                mask = (predictions_array == cls)
                weighted_votes[i] = np.sum(mask * self.weights_for_model[:, None], axis=0)

            aggregated_pred = unique_classes[np.argmax(weighted_votes, axis=0)]
        else:
            # Weighted average for regression
            weighted_sum = np.sum(predictions_array * self.weights_for_model[:, None], axis=0)
            aggregated_pred = weighted_sum / np.sum(self.weights_for_model)
        
        return aggregated_pred

    def aggregate_probabilities(self, probas_list):
        """
        Aggregate probabilities from multiple models with weights.

        Parameters:
        - probas_list: List of probability predictions from each model.

        Returns:
        - Aggregated probabilities.
        """
        probas_array = np.array(probas_list)
        
        # Weighted average for probabilities
        weighted_sum = np.sum(probas_array * self.weights_for_model[:, None, None], axis=0)
        aggregated_proba = weighted_sum / np.sum(self.weights_for_model)
        
        return aggregated_proba

# Example usage
if __name__ == "__main__":
    # Define task type and weights for models
    task_type = 'classification'
    weights = [0.4, 0.3, 0.2]

    # Instantiate the aggregator
    aggregator = ModelAggregator(task_type, weights)

    # Example predictions from 3 models (for classification)
    predictions_list = [
        [0, 1, 1, 0],
        [1, 1, 1, 0],
        [0, 0, 1, 0]
    ]

    # Aggregate predictions
    aggregated_predictions = aggregator.aggregate_predictions(predictions_list)
    print("Aggregated Predictions:", aggregated_predictions)

    # Example probability predictions from 3 models
    probas_list = [
        [[0.7, 0.3], [0.4, 0.6], [0.2, 0.8], [0.9, 0.1]],
        [[0.6, 0.4], [0.5, 0.5], [0.3, 0.7], [0.8, 0.2]],
        [[0.8, 0.2], [0.3, 0.7], [0.4, 0.6], [0.7, 0.3]]
    ]

    # Aggregate probabilities
    aggregated_probabilities = aggregator.aggregate_probabilities(probas_list)
    print("Aggregated Probabilities:", aggregated_probabilities)


Aggregated Predictions: [0 1 1 0]
Aggregated Probabilities: [[0.68888889 0.31111111]
 [0.41111111 0.58888889]
 [0.27777778 0.72222222]
 [0.82222222 0.17777778]]


In [27]:
['month', 'dayofyear', 'dayofweek', 'isweekend', 'couvrefeux', 'confinemenent',
'ramadan', 'bankHolidays', 'bankHolidaysEve', 'holidays', 'holidaysBorder',]

['month',
 'dayofyear',
 'dayofweek',
 'isweekend',
 'couvrefeux',
 'confinemenent',
 'ramadan',
 'bankHolidays',
 'bankHolidaysEve',
 'holidays',
 'holidaysBorder']

In [3]:
np.roll([1,2,3,4], shift=-1)

array([4, 1, 2, 3])

In [9]:
import itertools

kernel_size = 6

dims = [0,1]

number_of_simulation = (kernel_size // 2) + 1
combinaisons = list(itertools.product(dims, repeat=number_of_simulation))
combinaisons, len(combinaisons)

([(0, 0, 0, 0),
  (0, 0, 0, 1),
  (0, 0, 1, 0),
  (0, 0, 1, 1),
  (0, 1, 0, 0),
  (0, 1, 0, 1),
  (0, 1, 1, 0),
  (0, 1, 1, 1),
  (1, 0, 0, 0),
  (1, 0, 0, 1),
  (1, 0, 1, 0),
  (1, 0, 1, 1),
  (1, 1, 0, 0),
  (1, 1, 0, 1),
  (1, 1, 1, 0),
  (1, 1, 1, 1)],
 16)

In [28]:
array = np.moveaxis([[[  7, 213,   2,   0,   0,   0,   0,   0,   0,   1,   1 ]],
 [[  7, 213,   2,   0,   0,   0,   0,   0,   0,   1,   1 ]],
 [[  7, 213,   2,   0,   0,   0,   0,   0,   0,   1,   1 ]],
 [[  7, 213,   2,   0,   0,   0,   0,   0,   0,   1,   1 ]],
 [[  7, 213,   2,   0,   0,   0,   0,   0,   0,   1,   1 ]],
 [[  7, 213,   2,   0,   0,   0,   0,   0,   0,   1,   1 ]],
 [[  7, 213,   2,   0,   0,   0,   0,   0,   0,   1,   1 ]]], 1, 2).reshape(-1,11)

encoder.transform(array).values.reshape(-1, 1, 11)

array([[[32.07943786, 27.85498421, 10.70715742, 10.84466664,
         11.57700851, 11.15330096, 11.20806766, 10.93180603,
         10.93180603, 16.14544699, 15.51958359]],

       [[32.07943786, 27.85498421, 10.70715742, 10.84466664,
         11.57700851, 11.15330096, 11.20806766, 10.93180603,
         10.93180603, 16.14544699, 15.51958359]],

       [[32.07943786, 27.85498421, 10.70715742, 10.84466664,
         11.57700851, 11.15330096, 11.20806766, 10.93180603,
         10.93180603, 16.14544699, 15.51958359]],

       [[32.07943786, 27.85498421, 10.70715742, 10.84466664,
         11.57700851, 11.15330096, 11.20806766, 10.93180603,
         10.93180603, 16.14544699, 15.51958359]],

       [[32.07943786, 27.85498421, 10.70715742, 10.84466664,
         11.57700851, 11.15330096, 11.20806766, 10.93180603,
         10.93180603, 16.14544699, 15.51958359]],

       [[32.07943786, 27.85498421, 10.70715742, 10.84466664,
         11.57700851, 11.15330096, 11.20806766, 10.93180603,
         10.9